In [2]:
import pandas as pd
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
import numpy as np

In [8]:
label = pd.read_csv('../../data/comp5046-labels.csv',header=None)
Lr_label = list(set(label[2].values))


def prediction(x):
    tp = []
    for w in W:
        tp.append(x.dot(w))
    predict_num = tp.index(max(tp))
    return predict_num


folds = list(range(10))
text_len = 2700
num_of_epoch = 5


for fold in folds:
    step = (num_of_epoch*(2700))
    total_step = step
    W = [np.zeros(5608) for i in range(len(Lr_label))]
    a = [np.zeros(5608) for i in range(len(Lr_label))] # average w
    for epoch in range(num_of_epoch):
        number_of_training = 0
        predict_number_list = list(range(200 * fold, 200 * (fold + 1)))
        for text in pd.read_csv('result/NN_VB_data.csv',header=None,chunksize=600):
            label = pd.read_csv('../../data/comp5046-labels.csv',header=None)
            
            text = pd.merge(text,label[[1,2]],left_on=5608,right_on=1)
 
            y = [Lr_label.index(it) for it in text['2_y'].values]
   
            X = text.values[:,:-3]
            for each_data in range(len(X)):
                if number_of_training not in predict_number_list:
#                     print(number_of_training)
                    number_of_training+=1
                    y_ = prediction(X[each_data])
                    true_y = y[each_data]
#                     print(y_,true_y)
                    step -=1
                    if y_ != true_y:
#                         print(y_,true_y)
                       
                        W[true_y] = W[true_y] + X[each_data]
                        W[y_] = W[y_] - X[each_data] 
                        a[true_y] = a[true_y] + (X[each_data])*(step/(num_of_epoch*2700))
                        a[y_] = a[y_] - (X[each_data])*(step/(num_of_epoch*2700))
                        print('#'*(100-int(100*step/total_step))+'-'*int((step/total_step)*100)+' '+str(int(100-(step/total_step)*100))+'%',end='\r',flush=True)
#                     
                else:
                    number_of_training+=1
    W = a
    
    number_of_training = 0
    total_y_ = []
    total_y = []

    total_y_label = [Lr_label[i] for i in total_y]
    total_y_label_ = [Lr_label[i] for i in total_y_]
    predict_number_list = list(range(200 * fold, 200 * (fold + 1)))
    for text in pd.read_csv('result/NN_VB_data.csv',header=None,chunksize=600):
        label = pd.read_csv('../../data/comp5046-labels.csv',header=None)
        text = pd.merge(text,label[[1,2]],left_on=5608,right_on=1)

        y = [Lr_label.index(it) for it in text['2_y'].values]

        X = text.values[:,:-3]
        for each_data in range(len(X)):

            if number_of_training in predict_number_list and number_of_training < 200* (fold + 1):
                number_of_training+=1
                y_ = prediction(X[each_data])
                true_y = y[each_data]

                total_y_.append(y_)
                total_y.append(true_y)
            elif number_of_training not in predict_number_list and number_of_training < 200 * fold:
                number_of_training+=1
            elif number_of_training not in predict_number_list and number_of_training > 200 * (fold + 1):
                number_of_training+=1
                print('Testing is over.')
                break

        if len(total_y_) == 200: # Save the perfomance of result
            total_y_label = [Lr_label[i] for i in total_y]
            total_y_label_ = [Lr_label[i] for i in total_y_]
            print("Prediction score:\n%s" % metrics.classification_report(total_y_label, total_y_label_))
            result = metrics.classification_report(total_y_label, total_y_label_)
            macro = ' '.join(list(map(str,metrics.precision_recall_fscore_support(total_y_label,total_y_label_,average='macro'))))
            micro = ' '.join(list(map(str,metrics.precision_recall_fscore_support(total_y_label,total_y_label_,average='micro'))))
            with open('result_%s.csv' % fold, 'w') as file:  # Save the metrics result to file
                for line in result:
                    file.write(line)
#                 elapsed_time = time.time() - start_time
#                 file.write('Fold %s, time is %s. \n' % (fold, elapsed_time))  # Save the time cost to file
                file.write('\n macro: '+macro)
                file.write('\n micro: '+micro)
            print('Save the result.')
            break



/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Prediction score:
                       precision    recall  f1-score   support

            Education       0.81      0.93      0.87        14
        Entertainment       0.92      0.81      0.86        57
                Error       0.35      0.38      0.36        24
              Finance       1.00      0.50      0.67         2
               Health       0.00      0.00      0.00         0
          Hospitality       0.00      0.00      0.00         0
                Other       0.00      0.00      0.00         2
Public administration       0.52      0.61      0.56        23
          Real estate       0.00      0.00      0.00         2
            Resources       0.67      1.00      0.80         2
              SciTech       0.17      0.25      0.20         4
               Social       0.62      0.62      0.62        21
               Sports       0.75      0.80      0.77        30
                Trade       0.54      0.50      0.52        14
            Transport       0.67    